In [ ]:
# Greenhouse Gas Analytics - Data Cleaning and Preprocessing
# Notebook 02: Comprehensive Data Cleaning Pipeline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import re
warnings.filterwarnings('ignore')

# Set styling
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🧹 Greenhouse Gas Analytics - Data Cleaning")
print("="*50)

# ## 1. Load Raw Data

def load_raw_data():
    """Load raw data with error handling"""
    try:
        df = pd.read_csv('../data/raw/Methane_final.csv')
        print(f"✅ Raw data loaded successfully!")
        print(f"📊 Original shape: {df.shape}")
        return df
    except FileNotFoundError:
        print("❌ Raw data file not found. Creating sample data...")
        return create_sample_data()

def create_sample_data():
    """Create comprehensive sample dataset for demonstration"""
    np.random.seed(42)
    
    # Realistic country and region mapping
    countries_regions = {
        'China': 'Asia', 'India': 'Asia', 'United States': 'North America', 
        'Indonesia': 'Asia', 'Brazil': 'South America', 'Nigeria': 'Africa',
        'Russia': 'Europe', 'Mexico': 'North America', 'Iran': 'Asia',
        'Germany': 'Europe', 'Turkey': 'Europe', 'Canada': 'North America',
        'Australia': 'Oceania', 'Argentina': 'South America', 'Algeria': 'Africa',
        'Kazakhstan': 'Asia', 'Uzbekistan': 'Asia', 'Thailand': 'Asia',
        'Malaysia': 'Asia', 'Venezuela': 'South America', 'Saudi Arabia': 'Asia',
        'Pakistan': 'Asia', 'Egypt': 'Africa', 'Ukraine': 'Europe',
        'Bangladesh': 'Asia', 'Vietnam': 'Asia', 'Philippines': 'Asia',
        'Myanmar': 'Asia', 'Poland': 'Europe', 'South Africa': 'Africa'
    }
    
    # Emission types and realistic segments
    emission_data = {
        'Agriculture': ['Livestock', 'Rice Cultivation', 'Agricultural Waste', 'Enteric Fermentation'],
        'Energy': ['Oil & Gas', 'Coal Mining', 'Gas pipelines', 'Onshore oil', 'Bioenergy'],
        'Waste': ['Landfills', 'Wastewater Treatment', 'Solid Waste', 'Composting'],
        'Other': ['Industrial Processes', 'Transportation', 'Fugitive Emissions', 'Total']
    }
    
    base_years = ['2019-2021', '2020-2021', '2022', '2021', '2019', '2020']
    
    data = []
    for country, region in countries_regions.items():
        # Generate multiple records per country
        num_records = np.random.randint(15, 30)
        
        for _ in range(num_records):
            # Select emission type and appropriate segment
            emission_type = np.random.choice(list(emission_data.keys()))
            segment = np.random.choice(emission_data[emission_type])
            
            # Generate realistic emissions based on country and type
            base_emission = np.random.exponential(30) + np.random.normal(25, 15)
            
            # Add some data quality issues intentionally
            if np.random.random() < 0.02:  # 2% missing emissions
                emission_value = np.nan
            elif np.random.random() < 0.01:  # 1% negative values (errors)
                emission_value = -np.random.uniform(0, 10)
            else:
                emission_value = max(0, base_emission)
            
            # Add some inconsistencies in naming
            if np.random.random() < 0.05:  # 5% inconsistent country names
                country_name = country.upper() if np.random.random() < 0.5 else country.lower()
            else:
                country_name = country
            
            data.append({
                'region': region,
                'country': country_name,
                'emissions': emission_value,
                'type': emission_type,
                'segment': segment,
                'reason': 'All',
                'baseYear': np.random.choice(base_years)
            })
    
    # Add some completely null rows (1%)
    null_rows = int(len(data) * 0.01)
    for _ in range(null_rows):
        data.append({col: np.nan for col in ['region', 'country', 'emissions', 'type', 'segment', 'reason', 'baseYear']})
    
    return pd.DataFrame(data)

# Load the data
df_raw = load_raw_data()

print(f"\n📋 RAW DATA OVERVIEW:")
print("="*25)
print(f"Shape: {df_raw.shape}")
print(f"Columns: {list(df_raw.columns)}")
print(f"\nFirst 5 rows:")
print(df_raw.head())

# ## 2. Initial Data Quality Assessment

def assess_data_quality(df):
    """Comprehensive data quality assessment"""
    print(f"\n🔍 DATA QUALITY ASSESSMENT:")
    print("="*35)
    
    quality_report = {}
    
    # Missing values analysis
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    
    print(f"📊 MISSING VALUES:")
    for col in df.columns:
        if missing_data[col] > 0:
            print(f"  • {col}: {missing_data[col]} ({missing_percent[col]:.1f}%)")
    
    quality_report['missing_values'] = missing_data.to_dict()
    
    # Duplicate rows
    duplicates = df.duplicated().sum()
    print(f"\n🔄 DUPLICATES: {duplicates} rows")
    quality_report['duplicates'] = duplicates
    
    # Data type issues
    print(f"\n📝 DATA TYPES:")
    for col in df.columns:
        print(f"  • {col}: {df[col].dtype}")
    
    # Specific validations for emissions data
    if 'emissions' in df.columns:
        df['emissions_numeric'] = pd.to_numeric(df['emissions'], errors='coerce')
        
        negative_emissions = (df['emissions_numeric'] < 0).sum()
        zero_emissions = (df['emissions_numeric'] == 0).sum()
        
        print(f"\n⚠️  EMISSIONS VALIDATIONS:")
        print(f"  • Negative emissions: {negative_emissions}")
        print(f"  • Zero emissions: {zero_emissions}")
        
        quality_report['negative_emissions'] = negative_emissions
        quality_report['zero_emissions'] = zero_emissions
    
    # Categorical data inconsistencies
    categorical_cols = df.select_dtypes(include=['object']).columns
    print(f"\n🏷️  CATEGORICAL DATA:")
    
    for col in categorical_cols:
        if col in df.columns:
            unique_vals = df[col].nunique()
            sample_vals = df[col].dropna().unique()[:5]
            print(f"  • {col}: {unique_vals} unique values")
            print(f"    Sample: {list(sample_vals)}")
            
            # Check for case inconsistencies
            if df[col].dtype == 'object':
                original_count = df[col].nunique()
                standardized_count = df[col].str.strip().str.title().nunique()
                if original_count != standardized_count:
                    print(f"    ⚠️  Potential case inconsistencies detected")
    
    return quality_report

quality_report = assess_data_quality(df_raw)

# ## 3. Data Cleaning Pipeline

class DataCleaner:
    """Comprehensive data cleaning pipeline"""
    
    def __init__(self, df):
        self.df = df.copy()
        self.cleaning_log = []
        
    def log_step(self, step, details):
        """Log cleaning steps"""
        self.cleaning_log.append({
            'step': step,
            'details': details,
            'shape_after': self.df.shape,
            'timestamp': datetime.now()
        })
        print(f"✅ {step}: {details}")
    
    def clean_column_names(self):
        """Standardize column names"""
        original_cols = list(self.df.columns)
        
        # Strip whitespace and standardize
        self.df.columns = self.df.columns.str.strip().str.lower().str.replace(' ', '_')
        
        # Create mapping for display purposes
        col_mapping = dict(zip(original_cols, self.df.columns))
        
        self.log_step("Column Names", f"Standardized {len(original_cols)} column names")
        return col_mapping
    
    def handle_missing_values(self, strategy='smart'):
        """Handle missing values with smart strategies"""
        initial_missing = self.df.isnull().sum().sum()
        
        if strategy == 'smart':
            # Drop rows where all values are missing
            self.df = self.df.dropna(how='all')
            
            # Handle emissions column specifically
            if 'emissions' in self.df.columns:
                # Convert to numeric first
                self.df['emissions'] = pd.to_numeric(self.df['emissions'], errors='coerce')
                
                # For emissions, we can't meaningfully impute, so drop missing values
                emissions_missing_before = self.df['emissions'].isnull().sum()
                self.df = self.df.dropna(subset=['emissions'])
                emissions_dropped = emissions_missing_before - self.df['emissions'].isnull().sum()
                
                if emissions_dropped > 0:
                    print(f"  • Dropped {emissions_dropped} rows with missing emissions")
            
            # Handle categorical missing values
            categorical_cols = self.df.select_dtypes(include=['object']).columns
            for col in categorical_cols:
                missing_count = self.df[col].isnull().sum()
                if missing_count > 0:
                    # Fill with 'Unknown' for categorical variables
                    self.df[col] = self.df[col].fillna('Unknown')
                    print(f"  • Filled {missing_count} missing values in {col} with 'Unknown'")
        
        final_missing = self.df.isnull().sum().sum()
        self.log_step("Missing Values", f"Reduced from {initial_missing} to {final_missing} missing values")
    
    def standardize_categorical_values(self):
        """Standardize categorical values"""
        changes_made = 0
        
        categorical_cols = self.df.select_dtypes(include=['object']).columns
        
        for col in categorical_cols:
            if col in self.df.columns:
                original_unique = self.df[col].nunique()
                
                # Strip whitespace and standardize case
                self.df[col] = self.df[col].astype(str).str.strip()
                
                # Specific standardizations
                if col == 'country':
                    # Standardize country names
                    country_mapping = {
                        'united states': 'United States',
                        'usa': 'United States',
                        'us': 'United States',
                        'uk': 'United Kingdom',
                        'uae': 'United Arab Emirates'
                    }
                    
                    for old_name, new_name in country_mapping.items():
                        mask = self.df[col].str.lower() == old_name.lower()
                        if mask.any():
                            self.df.loc[mask, col] = new_name
                            changes_made += mask.sum()
                
                # Apply title case for most categorical columns
                if col not in ['baseyear', 'reason']:
                    self.df[col] = self.df[col].str.title()
                
                new_unique = self.df[col].nunique()
                if original_unique != new_unique:
                    print(f"  • {col}: {original_unique} → {new_unique} unique values")
                    changes_made += (original_unique - new_unique)
        
        self.log_step("Categorical Standardization", f"Made {changes_made} standardization changes")
    
    def handle_outliers(self, method='iqr', factor=1.5):
        """Handle outliers in emissions data"""
        if 'emissions' not in self.df.columns:
            return
        
        initial_count = len(self.df)
        
        if method == 'iqr':
            Q1 = self.df['emissions'].quantile(0.25)
            Q3 = self.df['emissions'].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - factor * IQR
            upper_bound = Q3 + factor * IQR
            
            # Identify outliers
            outliers_mask = (self.df['emissions'] < lower_bound) | (self.df['emissions'] > upper_bound)
            outliers_count = outliers_mask.sum()
            
            # Instead of removing, let's cap the values
            self.df.loc[self.df['emissions'] < lower_bound, 'emissions'] = lower_bound
            self.df.loc[self.df['emissions'] > upper_bound, 'emissions'] = upper_bound
            
            self.log_step("Outlier Handling", f"Capped {outliers_count} outliers using IQR method")
        
        # Handle negative emissions (clear data errors)
        negative_count = (self.df['emissions'] < 0).sum()
        if negative_count > 0:
            self.df = self.df[self.df['emissions'] >= 0]
            self.log_step("Negative Emissions", f"Removed {negative_count} rows with negative emissions")
    
    def remove_duplicates(self):
        """Remove duplicate rows"""
        initial_count = len(self.df)
        
        # Remove exact duplicates
        self.df = self.df.drop_duplicates()
        
        duplicates_removed = initial_count - len(self.df)
        
        if duplicates_removed > 0:
            self.log_step("Duplicate Removal", f"Removed {duplicates_removed} duplicate rows")
        else:
            self.log_step("Duplicate Check", "No duplicates found")
    
    def create_derived_features(self):
        """Create useful derived features"""
        features_created = 0
        
        # Extract year from baseYear
        if 'baseyear' in self.df.columns:
            # Handle different baseYear formats
            self.df['year'] = self.df['baseyear'].str.extract('(\d{4})', expand=False)
            self.df['year'] = pd.to_numeric(self.df['year'], errors='coerce')
            
            # For ranges like "2019-2021", take the end year
            range_mask = self.df['baseyear'].str.contains('-', na=False)
            if range_mask.any():
                range_years = self.df.loc[range_mask, 'baseyear'].str.extract('(\d{4})-(\d{4})')
                self.df.loc[range_mask, 'year'] = pd.to_numeric(range_years[1])  # Take end year
            
            features_created += 1
            print(f"  • Created 'year' from 'baseyear'")
        
        # Create emissions categories
        if 'emissions' in self.df.columns:
            # Categorize emissions into low, medium, high
            emissions_quantiles = self.df['emissions'].quantile([0.33, 0.67])
            
            def categorize_emissions(value):
                if pd.isna(value):
                    return 'Unknown'
                elif value <= emissions_quantiles.iloc[0]:
                    return 'Low'
                elif value <= emissions_quantiles.iloc[1]:
                    return 'Medium'
                else:
                    return 'High'
            
            self.df['emissions_category'] = self.df['emissions'].apply(categorize_emissions)
            features_created += 1
            print(f"  • Created 'emissions_category'")
        
        # Create regional groupings
        if 'region' in self.df.columns:
            # Group regions for analysis
            region_groups = {
                'Asia': 'Asia-Pacific',
                'Oceania': 'Asia-Pacific',
                'Europe': 'Europe',
                'North America': 'Americas',
                'South America': 'Americas',
                'Africa': 'Africa'
            }
            
            self.df['region_group'] = self.df['region'].map(region_groups).fillna('Other')
            features_created += 1
            print(f"  • Created 'region_group'")
        
        self.log_step("Feature Engineering", f"Created {features_created} derived features")
    
    def validate_cleaned_data(self):
        """Validate the cleaned dataset"""
        print(f"\n✅ DATA VALIDATION:")
        print("="*20)
        
        validation_results = {}
        
        # Check data completeness
        completeness = (1 - self.df.isnull().sum() / len(self.df)) * 100
        print(f"Data Completeness by Column:")
        for col in self.df.columns:
            print(f"  • {col}: {completeness[col]:.1f}%")
        
        validation_results['completeness'] = completeness.to_dict()
        
        # Check emissions data validity
        if 'emissions' in self.df.columns:
            emissions_stats = {
                'count': self.df['emissions'].count(),
                'mean': self.df['emissions'].mean(),
                'std': self.df['emissions'].std(),
                'min': self.df['emissions'].min(),
                'max': self.df['emissions'].max(),
                'negative_count': (self.df['emissions'] < 0).sum(),
                'zero_count': (self.df['emissions'] == 0).sum()
            }
            
            print(f"\nEmissions Data Validation:")
            print(f"  • Valid records: {emissions_stats['count']}")
            print(f"  • Range: {emissions_stats['min']:.2f} - {emissions_stats['max']:.2f}")
            print(f"  • Mean: {emissions_stats['mean']:.2f}")
            print(f"  • Negative values: {emissions_stats['negative_count']}")
            print(f"  • Zero values: {emissions_stats['zero_count']}")
            
            validation_results['emissions_stats'] = emissions_stats
        
        # Check categorical data consistency
        categorical_cols = self.df.select_dtypes(include=['object']).columns
        print(f"\nCategorical Data Summary:")
        for col in categorical_cols:
            unique_count = self.df[col].nunique()
            print(f"  • {col}: {unique_count} unique values")
            
            # Show top categories
            top_categories = self.df[col].value_counts().head(3)
            for category, count in top_categories.items():
                pct = (count / len(self.df)) * 100
                print(f"    - {category}: {count} ({pct:.1f}%)")
        
        # Overall data quality score
        avg_completeness = completeness.mean()
        has_negatives = validation_results.get('emissions_stats', {}).get('negative_count', 0) > 0
        
        if avg_completeness >= 95 and not has_negatives:
            quality_score = "Excellent"
        elif avg_completeness >= 90 and not has_negatives:
            quality_score = "Good"
        elif avg_completeness >= 80:
            quality_score = "Fair"
        else:
            quality_score = "Poor"
        
        print(f"\n🏆 Overall Data Quality: {quality_score}")
        print(f"📊 Average Completeness: {avg_completeness:.1f}%")
        
        validation_results['quality_score'] = quality_score
        validation_results['avg_completeness'] = avg_completeness
        
        return validation_results
    
    def get_cleaning_summary(self):
        """Get summary of cleaning operations"""
        return {
            'steps_performed': len(self.cleaning_log),
            'final_shape': self.df.shape,
            'cleaning_log': self.cleaning_log
        }

# ## 4. Execute Data Cleaning Pipeline

print(f"\n🧹 EXECUTING CLEANING PIPELINE:")
print("="*40)

# Initialize cleaner
cleaner = DataCleaner(df_raw)

# Execute cleaning steps
print(f"\n1️⃣ Cleaning column names...")
col_mapping = cleaner.clean_column_names()

print(f"\n2️⃣ Handling missing values...")
cleaner.handle_missing_values()

print(f"\n3️⃣ Standardizing categorical values...")
cleaner.standardize_categorical_values()

print(f"\n4️⃣ Handling outliers...")
cleaner.handle_outliers()

print(f"\n5️⃣ Removing duplicates...")
cleaner.remove_duplicates()

print(f"\n6️⃣ Creating derived features...")
cleaner.create_derived_features()

print(f"\n7️⃣ Validating cleaned data...")
validation_results = cleaner.validate_cleaned_data()

# Get the cleaned dataframe
df_clean = cleaner.df.copy()

# ## 5. Before/After Comparison

def create_comparison_plots(df_before, df_after):
    """Create before/after comparison visualizations"""
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Data Cleaning: Before vs After Comparison', fontsize=16, fontweight='bold')
    
    # 1. Data completeness comparison
    before_completeness = (1 - df_before.isnull().sum() / len(df_before)) * 100
    after_completeness = (1 - df_after.isnull().sum() / len(df_after)) * 100
    
    x_pos = np.arange(len(before_completeness))
    width = 0.35
    
    axes[0,0].bar(x_pos - width/2, before_completeness.values, width, 
                  label='Before', alpha=0.8, color='red')
    axes[0,0].bar(x_pos + width/2, after_completeness.values, width, 
                  label='After', alpha=0.8, color='green')
    axes[0,0].set_title('Data Completeness by Column')
    axes[0,0].set_ylabel('Completeness %')
    axes[0,0].set_xticks(x_pos)
    axes[0,0].set_xticklabels(before_completeness.index, rotation=45)
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Dataset size comparison
    sizes = ['Before Cleaning', 'After Cleaning']
    counts = [len(df_before), len(df_after)]
    colors = ['lightcoral', 'lightgreen']
    
    axes[0,1].bar(sizes, counts, color=colors, alpha=0.8)
    axes[0,1].set_title('Dataset Size Comparison')
    axes[0,1].set_ylabel('Number of Records')
    for i, v in enumerate(counts):
        axes[0,1].text(i, v + max(counts)*0.01, str(v), ha='center', fontweight='bold')
    
    # 3. Emissions distribution comparison (if available)
    if 'emissions' in df_before.columns and 'emissions' in df_after.columns:
        # Convert to numeric for comparison
        emissions_before = pd.to_numeric(df_before['emissions'], errors='coerce').dropna()
        emissions_after = pd.to_numeric(df_after['emissions'], errors='coerce').dropna()
        
        axes[0,2].hist(emissions_before, bins=30, alpha=0.7, label='Before', color='red', density=True)
        axes[0,2].hist(emissions_after, bins=30, alpha=0.7, label='After', color='green', density=True)
        axes[0,2].set_title('Emissions Distribution')
        axes[0,2].set_xlabel('Emissions (Mt CO₂e)')
        axes[0,2].set_ylabel('Density')
        axes[0,2].legend()
        axes[0,2].grid(True, alpha=0.3)
    
    # 4. Missing values heatmap - Before
    missing_before = df_before.isnull()
    if missing_before.any().any():
        sns.heatmap(missing_before.transpose(), cbar=True, cmap='Reds', 
                   ax=axes[1,0], xticklabels=False)
        axes[1,0].set_title('Missing Values Pattern - Before')
    else:
        axes[1,0].text(0.5, 0.5, 'No Missing Values', ha='center', va='center', 
                       transform=axes[1,0].transAxes, fontsize=12)
        axes[1,0].set_title('Missing Values Pattern - Before')
    
    # 5. Missing values heatmap - After
    missing_after = df_after.isnull()
    if missing_after.any().any():
        sns.heatmap(missing_after.transpose(), cbar=True, cmap='Greens', 
                   ax=axes[1,1], xticklabels=False)
        axes[1,1].set_title('Missing Values Pattern - After')
    else:
        axes[1,1].text(0.5, 0.5, 'No Missing Values', ha='center', va='center', 
                       transform=axes[1,1].transAxes, fontsize=12)
        axes[1,1].set_title('Missing Values Pattern - After')
    
    # 6. Data quality metrics
    metrics = ['Records', 'Completeness %', 'Unique Countries', 'Unique Regions']
    before_metrics = [
        len(df_before),
        before_completeness.mean(),
        df_before['country'].nunique() if 'country' in df_before.columns else 0,
        df_before['region'].nunique() if 'region' in df_before.columns else 0
    ]
    after_metrics = [
        len(df_after),
        after_completeness.mean(),
        df_after['country'].nunique() if 'country' in df_after.columns else 0,
        df_after['region'].nunique() if 'region' in df_after.columns else 0
    ]
    
    x_pos = np.arange(len(metrics))
    width = 0.35
    
    # Normalize metrics for comparison (except for the first one)
    normalized_before = [before_metrics[0]/1000] + [m for m in before_metrics[1:]]
    normalized_after = [after_metrics[0]/1000] + [m for m in after_metrics[1:]]
    
    axes[1,2].bar(x_pos - width/2, normalized_before, width, 
                  label='Before', alpha=0.8, color='red')
    axes[1,2].bar(x_pos + width/2, normalized_after, width, 
                  label='After', alpha=0.8, color='green')
    axes[1,2].set_title('Data Quality Metrics')
    axes[1,2].set_xticks(x_pos)
    axes[1,2].set_xticklabels(['Records\n(x1000)'] + metrics[1:], rotation=0)
    axes[1,2].legend()
    axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print(f"\n📊 CREATING COMPARISON VISUALIZATIONS...")
create_comparison_plots(df_raw, df_clean)

# ## 6. Data Quality Report

def generate_quality_report(df_before, df_after, cleaning_log):
    """Generate comprehensive data quality report"""
    
    report = {
        'cleaning_timestamp': datetime.now(),
        'original_data': {
            'shape': df_before.shape,
            'missing_values': df_before.isnull().sum().to_dict(),
            'duplicates': df_before.duplicated().sum(),
            'data_types': df_before.dtypes.to_dict()
        },
        'cleaned_data': {
            'shape': df_after.shape,
            'missing_values': df_after.isnull().sum().to_dict(),
            'duplicates': df_after.duplicated().sum(),
            'data_types': df_after.dtypes.to_dict()
        },
        'cleaning_operations': cleaning_log,
        'improvements': {}
    }
    
    # Calculate improvements
    original_completeness = (1 - df_before.isnull().sum().sum() / (df_before.shape[0] * df_before.shape[1])) * 100
    cleaned_completeness = (1 - df_after.isnull().sum().sum() / (df_after.shape[0] * df_after.shape[1])) * 100
    
    report['improvements'] = {
        'completeness_improvement': cleaned_completeness - original_completeness,
        'records_change': df_after.shape[0] - df_before.shape[0],
        'duplicates_removed': df_before.duplicated().sum() - df_after.duplicated().sum()
    }
    
    return report

# Generate quality report
quality_report = generate_quality_report(df_raw, df_clean, cleaner.cleaning_log)

print(f"\n📋 FINAL DATA QUALITY REPORT:")
print("="*35)
print(f"🕐 Cleaning completed at: {quality_report['cleaning_timestamp']}")
print(f"\n📊 Data Changes:")
print(f"  • Original shape: {quality_report['original_data']['shape']}")
print(f"  • Final shape: {quality_report['cleaned_data']['shape']}")
print(f"  • Records change: {quality_report['improvements']['records_change']:+d}")

print(f"\n✅ Quality Improvements:")
print(f"  • Completeness: {quality_report['improvements']['completeness_improvement']:+.1f}%")
print(f"  • Duplicates removed: {quality_report['improvements']['duplicates_removed']}")

print(f"\n🔧 Operations Performed:")
for i, operation in enumerate(cleaner.cleaning_log, 1):
    print(f"  {i}. {operation['step']}: {operation['details']}")

# ## 7. Export Cleaned Data

print(f"\n💾 EXPORTING CLEANED DATA...")
print("="*30)

# Create output directories if they don't exist
import os
os.makedirs('../data/processed', exist_ok=True)

# Export to multiple formats
export_formats = {
    'parquet': '../data/processed/cleaned_data.parquet',
    'csv': '../data/processed/cleaned_data.csv',
    'excel': '../data/processed/cleaned_data.xlsx'
}

for format_name, filepath in export_formats.items():
    try:
        if format_name == 'parquet':
            df_clean.to_parquet(filepath, index=False)
        elif format_name == 'csv':
            df_clean.to_csv(filepath, index=False)
        elif format_name == 'excel':
            df_clean.to_excel(filepath, index=False)
        
        print(f"✅ Exported to {format_name.upper()}: {filepath}")
        
    except Exception as e:
        print(f"❌ Failed to export {format_name}: {e}")

# Export quality report
try:
    import json
    with open('../data/processed/cleaning_report.json', 'w') as f:
        # Convert datetime and other non-serializable objects
        serializable_report = quality_report.copy()
        serializable_report['cleaning_timestamp'] = str(quality_report['cleaning_timestamp'])
        
        # Convert numpy types to Python types
        for key in ['original_data', 'cleaned_data']:
            for subkey in ['data_types']:
                serializable_report[key][subkey] = {
                    k: str(v) for k, v in serializable_report[key][subkey].items()
                }
        
        json.dump(serializable_report, f, indent=2, default=str)
    
    print(f"✅ Quality report exported: ../data/processed/cleaning_report.json")
except Exception as e:
    print(f"❌ Failed to export quality report: {e}")

# ## 8. Summary Statistics for Cleaned Data

print(f"\n📈 CLEANED DATA SUMMARY:")
print("="*30)

print(f"Final dataset shape: {df_clean.shape}")
print(f"Columns: {list(df_clean.columns)}")

if 'emissions' in df_clean.columns:
    emissions_summary = df_clean['emissions'].describe()
    print(f"\nEmissions Statistics:")
    for stat, value in emissions_summary.items():
        print(f"  • {stat}: {value:.2f}")

# Top categories
categorical_cols = df_clean.select_dtypes(include=['object']).columns
for col in categorical_cols[:3]:  # Show first 3 categorical columns
    print(f"\nTop 5 {col}:")
    top_values = df_clean[col].value_counts().head()
    for category, count in top_values.items():
        pct = (count / len(df_clean)) * 100
        print(f"  • {category}: {count} ({pct:.1f}%)")

# ## 9. Data Cleaning Recommendations

print(f"\n💡 RECOMMENDATIONS FOR FUTURE DATA COLLECTION:")
print("="*55)

recommendations = [
    "Standardize country name formats across all data sources",
    "Implement data validation rules to prevent negative emissions",
    "Establish consistent date/year formatting (YYYY or YYYY-YYYY)",
    "Create data dictionaries for categorical variables",
    "Implement regular data quality checks during collection",
    "Add data source metadata for traceability",
    "Consider implementing automated outlier detection",
    "Establish minimum required fields to reduce missing data"
]

for i, recommendation in enumerate(recommendations, 1):
    print(f"  {i}. {recommendation}")

print(f"\n🎉 DATA CLEANING COMPLETE!")
print("="*30)
print(f"✨ Clean dataset ready for analysis and visualization!")
print(f"📊 {df_clean.shape[0]} records across {df_clean.shape[1]} features")
print(f"🎯 Data quality: {validation_results.get('quality_score', 'Unknown')}")